# Combinando Datasets: Merge y Join

Una característica esencial que ofrece Pandas son sus operaciones join y merge de alto rendimiento en memoria.
Si alguna vez has trabajado con bases de datos, deberías estar familiarizado con este tipo de interacción de datos.
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo puede funcionar en la práctica.


In [ ]:
import pandas as pd
import numpy as np
from utils.funciones import Display # una función creada anteriormente en la carpeta funciones de utils

In [ ]:
# Para hacer merge hay que mezclar los mismos tipos de cosa EJ) no puedes unir dni y nombres porque no es lo mismo
# Con dataframe se hace generalmente merge, join no se hace casi

## Álgebra relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales, y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
El punto fuerte del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los bloques de construcción de operaciones más complicadas sobre cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado de forma eficiente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.


## Categorías de Joins

La función ``pd.merge()`` implementa varios tipos de uniones: las uniones *uno-a-uno*, *muchos-a-uno* y *muchos-a-muchos*.
A los tres tipos de uniones se accede mediante una llamada idéntica a la interfaz ``pd.merge()``; el tipo de unión realizada depende de la forma de los datos de entrada.
Aquí mostraremos ejemplos sencillos de los tres tipos de uniones, y discutiremos las opciones detalladas más adelante.

### Joins individuales

Tal vez el tipo más sencillo de expresión de fusión sea la unión uno a uno, que en muchos aspectos es muy similar a la concatenación por columnas.

Como ejemplo concreto, consideremos los dos ``DataFrame`` siguientes, que contienen información sobre varios empleados de una empresa:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
# Si los nombres no tienen el mismo orden, al concatenar se lia todo
Display('df1', 'df2', context=locals())

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos utilizar la función ``pd.merge()``:

In [ ]:
df3 = pd.merge(df1, df2) # El merge arregla ese lío
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna "empleado", y une automáticamente utilizando esta columna como clave.
El resultado de la fusión es un nuevo ``DataFrame`` que combina la información de las dos entradas.
Observa que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "empleado" difiere entre ``df1`` y ``df2``, y la función ``pd.merge()`` lo tiene en cuenta correctamente.
Además, hay que tener en cuenta que la fusión en general descarta el índice, excepto en el caso especial de las fusiones por índice (las palabras clave ``left_index`` y ``right_index``, de las que hablaremos más adelante).

### Joins muchos-a-uno

Los joins muchos-a-uno son uniones en las que una de las dos columnas clave contiene entradas duplicadas.
Para el caso muchos-a-uno, el ``DataFrame`` resultante conservará esas entradas duplicadas según corresponda.

In [4]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Alex', 'Steve']})
Display('df3', 'df4', 'pd.merge(df3, df4)', context=locals())

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering       Alex
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012       Alex
2     Lisa  Engineering       2004       Alex
3      Sue           HR       2014      Steve

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en uno o más lugares según lo requieran las entradas.

### Joins muchos-a-muchos

Los joins muchos-a-muchos son un poco confusas conceptualmente, pero sin embargo están bien definidas.
Si la columna clave tanto de la matriz izquierda como de la derecha contiene duplicados, el resultado es una fusión de muchos a muchos.
Quizá esto quede más claro con un ejemplo concreto.
Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.
Realizando una unión de muchos a muchos, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [5]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
Display('df1', 'df5', "pd.merge(df1, df5)", context=locals())

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.
Pero en la práctica, los conjuntos de datos raramente están tan limpios como con el que estamos trabajando aquí.

## Especificación de la clave de fusión

Ya hemos visto el comportamiento por defecto de ``pd.merge()``: busca uno o más nombres de columna que coincidan entre las dos entradas y los utiliza como clave.
Sin embargo, a menudo los nombres de las columnas no coinciden tan bien, y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

Lo más sencillo es especificar explícitamente el nombre de la columna clave mediante la palabra clave ``on``, que recibe un nombre de columna o una lista de nombres de columna:

In [6]:
Display('df1', 'df2', "pd.merge(df1, df2, on='employee')", context=locals())

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### Las palabras clave ``left_on`` y ``right_on``

En ocasiones, es posible que desee combinar dos conjuntos de datos con nombres de columna diferentes; por ejemplo, podemos tener un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".
En este caso, podemos utilizar las palabras clave ``left_on`` y ``right_on`` para especificar los dos nombres de columna:

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
Display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")', context=locals())
# Le estoy diciendo que name y employee son lo mismo aunque estén nombrado de diferente manera, 
# te da las dos columnas y luego tu ya decides con cual quedarte

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, utilizando el método ``drop()`` de ``DataFrame``:

In [7]:
# pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)
pd.merge(df1, df3, left_on="employee", right_on="name").drop(columns='name')

KeyError: 'name'

### Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de fusionar en una columna, se desea hacerlo en un índice.
Por ejemplo, tus datos podrían tener este aspecto:

In [ ]:
df1a = df1.set_index('employee') # Employee va a ser el índice pero no lo muestra por orden
df2a = df2.set_index('employee') 
Display('df1a', 'df2a', context=locals())

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Puedes utilizar el índice como clave para la fusión especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [11]:
aa = pd.merge(df1a, df2a, left_index=True, right_index=True)

In [ ]:
# Loc buscamos por el nombre
aa.loc['Jake'] # ESTO ES IMPORTANTE (el loc) por lo que te muestra

group        Engineering
hire_date           2012
Name: Jake, dtype: object

In [ ]:
aa.iloc[1] # Buscamos por posición

group        Engineering
hire_date           2012
Name: Jake, dtype: object

In [ ]:
Display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)", context=locals())
# Devulve un solo indice ordenado por orden alfabetico y luego las dos columnas
# Aqui el indice se llama employee, es decir que tiene nombre (aunque generalmente los indices no lo tengan)

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Por comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto une los índices:

In [15]:
Display('df1a', 'df2a', 'df1a.join(df2a)', context=locals()) # El join trabaja unica y exclusivamente por indice

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Si quieres mezclar índices y columnas, puedes combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [ ]:
Display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')", context=locals()) # Esto no debería de dar erro

KeyError: 'name'

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para más información sobre esto, consulte la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## Especificación de la aritmética de conjuntos para joins

En todos los ejemplos anteriores hemos pasado por alto una consideración importante a la hora de realizar una unión: el tipo de aritmética de conjuntos utilizado en la unión.
Esto ocurre cuando un valor aparece en una columna clave pero no en la otra. Veamos este ejemplo:

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
Display('df6', 'df7', 'pd.merge(df6, df7)', context=locals()) # Solo te muestra lo común (como el inner)

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos fusionado dos conjuntos de datos que sólo tienen una entrada de "name" en común: María.
Por defecto, el resultado contiene la *intersección* de los dos conjuntos de entradas; esto es lo que se conoce como *inner join*.
Podemos especificarlo explícitamente mediante la palabra clave ``how``, que por defecto es ``"inner"``:

In [ ]:
pd.merge(df6, df7, how='inner') # es lo mismo que lo de arriba

,name,food,drink
0,Mary,bread,wine


Otras opciones para la palabra clave ``how`` son ``'outer``, ``'left`` y ``'right``.
Un *outer join* devuelve un join sobre la unión de las columnas de entrada, y rellena todos los valores perdidos con NAs:

In [ ]:
Display('df6', 'df7', "pd.merge(df6, df7, how='outer')", context=locals()) # Aqui te lo muetsra todo

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0  Joseph    NaN  beer
1    Mary  bread  wine
2    Paul  beans   NaN
3   Peter   fish   NaN

Las opciones *left join* y *right join* devuelven uniones sobre las entradas de la izquierda y la derecha, respectivamente.
Por ejemplo:

In [ ]:
Display('df6', 'df7', "pd.merge(df6, df7, how='left')", context=locals()) 
# devuelve todo loq ue se puede utilizar en la izquierda mas la intersección de la derecha

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [ ]:
Display('df6', 'df7', "pd.merge(df6, df7, how='right')", context=locals()) # Te muestra los de la derecha

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

Las filas de salida corresponden ahora a las entradas de la entrada izquierda. Utilizando
``how='right'`` funciona de forma similar.

## Solapamiento de nombres de columna: La palabra clave ``suffixes``

Por último, puede darse el caso de que sus dos ``DataFrame`` de entrada tengan nombres de columna contradictorios.
Considere este ejemplo:

In [22]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
Display('df8', 'df9', 'pd.merge(df8, df9, on="name")', context=locals())

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Dado que la salida tendría dos nombres de columna en conflicto, la función de fusión añade automáticamente un sufijo ``_x`` o ``_y`` para hacer que las columnas de salida sean únicas.
Si estos valores por defecto no son apropiados, es posible especificar un sufijo personalizado utilizando la palabra clave ``suffixes``:

In [ ]:
Display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])', context=locals()) 
# podemos escribir los sufijos que queremos, sino panda te pone los que quiera

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

In [27]:
ab = pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])

In [29]:
for i in ab.columns:
    if '_L' in i:
        ab.drop(columns = i)

ab

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Estos sufijos funcionan en cualquiera de los posibles patrones de unión, y funcionan también si hay varias columnas superpuestas.

## Ejemplo: Datos de los Estados de EE.UU.

Las operaciones de fusión y unión son más frecuentes cuando se combinan datos de distintas fuentes.
Aquí veremos un ejemplo de datos sobre los estados de EE.UU. y su población.
Los archivos de datos se encuentran en http://github.com/jakevdp/data-USstates/:

In [25]:
# # Descargar los archivos dentro de la carpeta data
!curl -o data/state-population.csv https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -o data/state-areas.csv https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -o data/state-abbrevs.csv https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 57935  100 57935    0     0   176k      0 --:--:-- --:--:-- --:--:--  179k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   835  100   835    0     0   4912      0 --:--:-- --:--:-- --:--:--  5030
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   872  100   872    0     0   3885      0 --:--:-- --:--:-- --:--:--  3945
100   872  100   872    0     0   3879      0 --:

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [26]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

Display('pop.head()', 'areas.head()', 'abbrevs.head()', context=locals())

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Con esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE.UU. según su densidad de población en 2010.
Es evidente que disponemos de los datos necesarios para obtener este resultado, pero tendremos que combinar los conjuntos de datos para obtenerlo.

Empezaremos con una combinación múltiple que nos dará el nombre completo del estado dentro del ``DataFrame`` de población.
Queremos hacer la combinación basándonos en la columna "estado/región" de "pop" y en la columna "abreviatura" de "abbrevs".
Utilizaremos ``how='outer'`` para asegurarnos de que no se pierde ningún dato por no coincidir las etiquetas.

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer', # Outer!
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop(columns='abbreviation') # eliminar información duplicada en este caso la columna abreviaturas
merged.head() # Head muestra solo los 5 primeros (o el numero que pongas en el parentesis)

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


Comprobemos si se ha producido alguna discordancia, lo que podemos hacer buscando filas con nulos:

In [31]:
merged.isnull() # No hay nulos (a simple vita)

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,False
2540,False,False,False,False,False
2541,False,False,False,False,False
2542,False,False,False,False,False


In [32]:
merged.isnull().any() # Devuelve true donde hay nulos

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Algunos de los datos de ``población`` son nulos; ¡averigüemos cuáles son!

In [ ]:
# filtar con una máscara cuantos nulos tengo
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

Y lo que es más importante, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [ ]:
# Solo quiero ver solo el state/region y con unique solo los valores, no te los repite
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [ ]:
abbrevs # No pararece ni usa ni pr

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [36]:
pop['state/region'].unique() # 

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR', 'USA'],
      dtype=object)

Podemos deducir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (USA), mientras que estas entradas no aparecen en la clave de abreviatura del estado.
Podemos solucionarlo rápidamente introduciendo las entradas adecuadas:

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico' # Donde state/region sea PR pon puerto rico
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any() # ¿Hay más nulos? si si sale true

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Se acabaron los nulos en la columna ``state``: ¡listo!

Ahora podemos fusionar el resultado con los datos del área mediante un procedimiento similar.
Al examinar nuestros resultados, querremos unirlos en la columna ``state`` de ambos:

In [38]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


De nuevo, vamos a comprobar si hay nulos para ver si hay algún desajuste:

In [39]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Hay nulos en la columna ``area``; podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [40]:
final['state'][final['area (sq. mi)'].isnull()].unique() # cuanto nulos hay en area cuadrada

array(['United States'], dtype=object)

In [ ]:
areas # No aparece el area de eeuu, abria que sumar la de todos los estados para conseguirla

Vemos que nuestro ``DataFrame`` de ``areas`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra investigación actual:

In [41]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [42]:
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacerlo rápidamente (esto requiere tener instalado el paquete ``numexpr``.

In [ ]:
# Query optimiza su codigo
# Query es consulta (para consultar) filtra dependiendo de la condicion que tu le des
data2010 = final.query("year == 2010 & ages == 'total'") # Año = 2010, Años = total
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


Ahora vamos a calcular la densidad de población y mostrarla por orden.
Empezaremos por volver a indexar nuestros datos en el estado y, a continuación, calcularemos el resultado:

In [46]:
data2010.set_index('state', inplace=True) # que los estados sean los indices
density = data2010['population'] / data2010['area (sq. mi)'] # población entre area = densidad

In [ ]:
density.sort_values(ascending=False, inplace=True) # Ordena valores de forma descendente (porq pone ascendente false)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

El resultado es una clasificación de los estados de EE.UU. más Washington DC, y Puerto Rico por orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [ ]:
density.tail() # Devulve los ultimos 5

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de fusión de datos desordenados es una tarea habitual cuando se intenta responder a preguntas utilizando fuentes de datos del mundo real.

<!--NAVIGATION-->
< [Concatenado](5-Concatenado.ipynb) | [Agregación y agrupación](7-Agregaciones_y_agrupaciones.ipynb) >
